In [4]:
# !wget https://huggingface.co/datasets/shareAI/ShareGPT-Chinese-English-90k/resolve/main/sharegpt_jsonl/common_zh_70k.jsonl
# !wget https://huggingface.co/datasets/shareAI/ShareGPT-Chinese-English-90k/resolve/main/sharegpt_jsonl/common_en_70k.jsonl
# !wget https://huggingface.co/datasets/shareAI/ShareGPT-Chinese-English-90k/resolve/main/sharegpt_jsonl/unknow_zh_38k.jsonl
# !wget https://huggingface.co/datasets/shareAI/ShareGPT-Chinese-English-90k/resolve/main/sharegpt_jsonl/unknow_zh_38k_continue.jsonl
# !wget https://huggingface.co/datasets/shareAI/ShareGPT-Chinese-English-90k/resolve/main/sharegpt_jsonl/computer_en_26k_continue.jsonl
# !wget https://huggingface.co/datasets/shareAI/ShareGPT-Chinese-English-90k/resolve/main/sharegpt_jsonl/computer_zh_26k\(fixed\).jsonl
# !wget https://huggingface.co/datasets/FreedomIntelligence/alpaca-gpt4-chinese/resolve/main/alpaca-gpt4-chinese.json

In [5]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

system = [
    'always reply in mandarin',
    'sentiasa respond dalam bahasa mandarin',
    'sentiasa jawab in mandarin',
    'awak adalah chatbot yang respond dalam bahasa mandarin',
    'you are a chatbot that always respond in mandarin',
    'u always reply in mandarin',
    'awak sentiasa reply in mandarin',
    'you are a chatbot that always reply in mandarin',
    'you are an assistant that always reply in mandarin',
    'awak adalah chatbot yang sentiasa respond dalam mandarin',
    'awak adalah pembantu AI yang reply in mandarin',
    '你是一个总是用普通话回复的聊天机器人',
]

In [9]:
import json
import random

instructions = []

files = [
    'common_zh_70k.jsonl',
    'computer_zh_26k(fixed).jsonl'
]

for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            results = []
            for l_ in l['conversation']:
                results.extend([
                    f"<manusia>: {l_['human']}",
                    f"<bot>: {l_['assistant']}"
                ])
            if len(results) % 2 != 0:
                results = results[:-1]
            results[0] = results[0].replace('<manusia>:', f'<manusia>:{random.choice(system)}\n\n')
            instructions.append({
                'prompt_input': None,
                'input': '\n'.join(results),
                'output': None,
            })
            
len(instructions)

96842

In [11]:
from tqdm import tqdm

with open('alpaca-gpt4-chinese.json') as fopen:
    data = json.load(fopen)
    
for d in tqdm(data):
    l = d['conversations'][0]['value']
    r = d['conversations'][1]['value']
    results =[
        f"<manusia>: {l}",
        f"<bot>: {r}"
    ]
    results[0] = results[0].replace('<manusia>:', f'<manusia>:{random.choice(system)}\n\n')
    instructions.append({
        'prompt_input': None,
        'input': '\n'.join(results),
        'output': None,
    })
    
len(instructions)

100%|█████████████████████████████████| 49969/49969 [00:00<00:00, 857770.93it/s]


146811

In [19]:
import pandas as pd

pd.DataFrame(instructions).to_parquet('prepared-force-mandarin.parquet')

In [20]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-force-mandarin.parquet",
    path_in_repo='data/force_mandarin-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-force-mandarin.parquet:   0%|          | 0.00/181M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/be8aeb93e070086ea6efefe776ac5bba5741e51a', commit_message='Upload data/force_mandarin-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='be8aeb93e070086ea6efefe776ac5bba5741e51a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-SFT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-SFT'), pr_revision=None, pr_num=None)